In [1]:
import math
def comparison(testlabel, resultslabel):

    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for row1 in range(len(resultslabel)):
        if resultslabel[row1] < 0.5:
            resultslabel[row1] = 0
        else:
            resultslabel[row1] = 1

    for row2 in range(len(testlabel)):

        if testlabel[row2] == 1 and testlabel[row2] == resultslabel[row2]:
            TP = TP + 1
        if testlabel[row2] == 0 and testlabel[row2] != resultslabel[row2]:
            FP = FP + 1
        if testlabel[row2] == 0 and testlabel[row2] == resultslabel[row2]:
            TN = TN + 1
        if testlabel[row2] == 1 and testlabel[row2] != resultslabel[row2]:
            FN = FN + 1

    # ACC：accuracy
    if TP + TN + FP + FN != 0:
        ACC = (TP + TN) / (TP + TN + FP + FN)
    else:
        ACC = 999999

    # F1 score：is the harmonic mean of precision and sensitivity
    if TP + FP + FN != 0:
        F1 = (2 * TP) / (2 * TP + FP + FN)
    else:
        F1 = 999999

    # MCC：Matthews correlation coefficient
    if (TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) != 0:
        MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    else:
        MCC = 999999

    return TP, FP, TN, FN, ACC, F1, MCC

In [2]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,0,1])
        elif (j == 'T'):
            dd.append([0,0,0,1,0])
        elif (j == 'G'):
            dd.append([0,0,1,0,0])
        elif (j == 'C'):
            dd.append([0,1,0,0,0])
        else:
            dd.append([0,0,0,0,0])
    while(len(dd) < 500):
        dd.append([0,0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,5),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



2023-09-19 03:55:28.904615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(40544, 500, 5)
(4505, 500, 5)


2023-09-19 03:56:18.150671: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23797 MB memory:  -> device: 0, name: Tesla P40, pci bus id: 0000:04:00.0, compute capability: 6.1


Epoch 1/20


2023-09-19 03:56:21.800807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2023-09-19 03:56:22.293474: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f6ce81fe6d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-19 03:56:22.293523: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-09-19 03:56:22.301295: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-19 03:56:22.521763: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

571/571 [==============================] - 6s 11ms/step - loss: 0.2878 - accuracy: 0.8853 - val_loss: 0.2629 - val_accuracy: 0.8994 - lr: 0.0010
Epoch 6/20
566/571 [============================>.] - ETA: 0s - loss: 0.2818 - accuracy: 0.8893INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 11ms/step - loss: 0.2821 - accuracy: 0.8890 - val_loss: 0.2548 - val_accuracy: 0.9063 - lr: 0.0010
Epoch 7/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2784 - accuracy: 0.8894 - val_loss: 0.2606 - val_accuracy: 0.8984 - lr: 0.0010
Epoch 8/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2746 - accuracy: 0.8901 - val_loss: 0.2689 - val_accuracy: 0.8927 - lr: 0.0010
Epoch 9/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2648 - accuracy: 0.8965 - val_loss: 0.2614 - val_accuracy: 0.8991 - lr: 0.0010
Epoch 10/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2637 - accuracy: 0.8963 - val_loss: 0.2588 - val_accuracy: 0.8979 - lr: 0.0010
Epoch 11/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2658 - accuracy: 0.8952 - val_loss: 0.2629 - val_accuracy: 0.8982 - lr: 0.0010
Epoch 12/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2

2023-09-19 03:58:07.900342: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 1s 3ms/step
PLeccNET
TP: 2223 FP: 283 TN: 1794 FN: 205
ACC: 0.8916759156492786 F1: 0.9010944466963924 MCC: 0.7819126230229125


In [3]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'T'):
            dd.append([0,0,1,0])
        elif (j == 'G'):
            dd.append([0,0,1,1])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < 500):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,4),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(40544, 500, 4)
(4505, 500, 4)
Epoch 1/20
568/571 [============================>.] - ETA: 0s - loss: 0.5193 - accuracy: 0.7240INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 8s 11ms/step - loss: 0.5189 - accuracy: 0.7244 - val_loss: 0.3330 - val_accuracy: 0.8691 - lr: 0.0010
Epoch 2/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3501 - accuracy: 0.8490 - val_loss: 0.3380 - val_accuracy: 0.8560 - lr: 0.0010
Epoch 3/20
565/571 [============================>.] - ETA: 0s - loss: 0.3208 - accuracy: 0.8671INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.3206 - accuracy: 0.8671 - val_loss: 0.2862 - val_accuracy: 0.8890 - lr: 0.0010
Epoch 4/20
569/571 [============================>.] - ETA: 0s - loss: 0.3084 - accuracy: 0.8721INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


...

571/571 [==============================] - 6s 10ms/step - loss: 0.2834 - accuracy: 0.8851 - val_loss: 0.2557 - val_accuracy: 0.9028 - lr: 0.0010
Epoch 12/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2840 - accuracy: 0.8855 - val_loss: 0.2655 - val_accuracy: 0.8989 - lr: 0.0010
Epoch 13/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2823 - accuracy: 0.8868 - val_loss: 0.2548 - val_accuracy: 0.9023 - lr: 0.0010
Epoch 14/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2709 - accuracy: 0.8927 - val_loss: 0.2524 - val_accuracy: 0.9011 - lr: 0.0010
Epoch 15/20
568/571 [============================>.] - ETA: 0s - loss: 0.2735 - accuracy: 0.8922INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.2733 - accuracy: 0.8924 - val_loss: 0.2504 - val_accuracy: 0.9051 - lr: 0.0010
Epoch 16/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2703 - accuracy: 0.8917 - val_loss: 0.2599 - val_accuracy: 0.8979 - lr: 0.0010
Epoch 17/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2664 - accuracy: 0.8920 - val_loss: 0.2485 - val_accuracy: 0.9023 - lr: 0.0010
Epoch 18/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2664 - accuracy: 0.8941 - val_loss: 0.2537 - val_accuracy: 0.8996 - lr: 0.0010
Epoch 19/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2678 - accuracy: 0.8945 - val_loss: 0.3303 - val_accuracy: 0.8565 - lr: 0.0010
Epoch 20/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2643 - accuracy: 0.8941 - val_loss: 0.2687 - val_accuracy: 0.8922 - lr: 0.0010


2023-09-19 04:14:14.311070: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 1s 3ms/step
PLeccNET
TP: 2235 FP: 271 TN: 1786 FN: 213
ACC: 0.8925638179800222 F1: 0.9023011707710941 MCC: 0.7832729659551647


In [4]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,1])
        elif (j == 'T'):
            dd.append([0,1,0])
        elif (j == 'G'):
            dd.append([1,0,0])
        elif (j == 'C'):
            dd.append([-1,-1,-1])
        else:
            dd.append([0,0,0])
    while(len(dd) < 500):
        dd.append([0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,3),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(40544, 500, 3)
(4505, 500, 3)
Epoch 1/20
571/571 [==============================] - ETA: 0s - loss: 0.4715 - accuracy: 0.7639INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 8s 11ms/step - loss: 0.4715 - accuracy: 0.7639 - val_loss: 0.3125 - val_accuracy: 0.8727 - lr: 0.0010
Epoch 2/20
566/571 [============================>.] - ETA: 0s - loss: 0.3143 - accuracy: 0.8719INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.3140 - accuracy: 0.8720 - val_loss: 0.2665 - val_accuracy: 0.8964 - lr: 0.0010
Epoch 3/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3079 - accuracy: 0.8754 - val_loss: 0.3109 - val_accuracy: 0.8718 - lr: 0.0010
Epoch 4/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2903 - accuracy: 0.8856 - val_loss: 0.2685 - val_accuracy: 0.8949 - lr: 0.0010
Epoch 5/20
567/571 [============================>.] - ETA: 0s - loss: 0.2845 - accuracy: 0.8879INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


...

571/571 [==============================] - 6s 10ms/step - loss: 0.2840 - accuracy: 0.8881 - val_loss: 0.2531 - val_accuracy: 0.9011 - lr: 0.0010
Epoch 6/20
565/571 [============================>.] - ETA: 0s - loss: 0.2806 - accuracy: 0.8891INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.2811 - accuracy: 0.8887 - val_loss: 0.2499 - val_accuracy: 0.9058 - lr: 0.0010
Epoch 7/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2773 - accuracy: 0.8908 - val_loss: 0.2821 - val_accuracy: 0.8910 - lr: 0.0010
Epoch 8/20
571/571 [==============================] - 5s 9ms/step - loss: 0.2773 - accuracy: 0.8906 - val_loss: 0.2536 - val_accuracy: 0.9048 - lr: 0.0010
Epoch 9/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2711 - accuracy: 0.8944 - val_loss: 0.3222 - val_accuracy: 0.8654 - lr: 0.0010
Epoch 10/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2696 - accuracy: 0.8959 - val_loss: 0.2776 - val_accuracy: 0.8885 - lr: 0.0010
Epoch 11/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2635 - accuracy: 0.8979 - val_loss: 0.2600 - val_accuracy: 0.8979 - lr: 0.0010
Epoch 12/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2

2023-09-19 04:33:41.889218: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 0s 3ms/step
PLeccNET
TP: 2232 FP: 274 TN: 1779 FN: 220
ACC: 0.8903440621531632 F1: 0.9003630496167809 MCC: 0.7787084316430557


In [5]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,1])
        elif (j == 'T'):
            dd.append([0,0,2])
        elif (j == 'G'):
            dd.append([0,1,0])
        elif (j == 'C'):
            dd.append([0,1,1])
        else:
            dd.append([0,0,0])
    while(len(dd) < 500):
        dd.append([0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,3),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(40544, 500, 3)
(4505, 500, 3)
Epoch 1/20
567/571 [============================>.] - ETA: 0s - loss: 0.5084 - accuracy: 0.7421INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 8s 11ms/step - loss: 0.5077 - accuracy: 0.7424 - val_loss: 0.4975 - val_accuracy: 0.7596 - lr: 0.0010
Epoch 2/20
565/571 [============================>.] - ETA: 0s - loss: 0.3826 - accuracy: 0.8384INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.3819 - accuracy: 0.8387 - val_loss: 0.3057 - val_accuracy: 0.8838 - lr: 0.0010
Epoch 3/20
567/571 [============================>.] - ETA: 0s - loss: 0.3524 - accuracy: 0.8499INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


...

571/571 [==============================] - 6s 10ms/step - loss: 0.3036 - accuracy: 0.8751 - val_loss: 0.2733 - val_accuracy: 0.8932 - lr: 0.0010
Epoch 8/20
568/571 [============================>.] - ETA: 0s - loss: 0.3064 - accuracy: 0.8733INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.3067 - accuracy: 0.8732 - val_loss: 0.2725 - val_accuracy: 0.8952 - lr: 0.0010
Epoch 9/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2959 - accuracy: 0.8796 - val_loss: 0.3239 - val_accuracy: 0.8651 - lr: 0.0010
Epoch 10/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2931 - accuracy: 0.8822 - val_loss: 0.3223 - val_accuracy: 0.8607 - lr: 0.0010
Epoch 11/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2827 - accuracy: 0.8869 - val_loss: 0.2987 - val_accuracy: 0.8710 - lr: 0.0010
Epoch 12/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2806 - accuracy: 0.8884 - val_loss: 0.3419 - val_accuracy: 0.8491 - lr: 0.0010
Epoch 13/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2827 - accuracy: 0.8871 - val_loss: 0.2661 - val_accuracy: 0.8925 - lr: 0.0010
Epoch 14/20
571/571 [==============================] - 5s 8ms/step - loss: 0

2023-09-19 04:36:46.737065: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 0s 3ms/step
PLeccNET
TP: 2219 FP: 287 TN: 1728 FN: 271
ACC: 0.8761376248612652 F1: 0.888310648518815 MCC: 0.7493197566540749


In [6]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,1])
        elif (j == 'T'):
            dd.append([0,1,0])
        elif (j == 'G'):
            dd.append([1,0,0])
        elif (j == 'C'):
            dd.append([0,0,0])
        else:
            dd.append([0,0,0])
    while(len(dd) < 500):
        dd.append([0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,3),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(40544, 500, 3)
(4505, 500, 3)
Epoch 1/20
566/571 [============================>.] - ETA: 0s - loss: 0.4532 - accuracy: 0.7968INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 8s 11ms/step - loss: 0.4530 - accuracy: 0.7969 - val_loss: 0.3708 - val_accuracy: 0.8439 - lr: 0.0010
Epoch 2/20
571/571 [==============================] - ETA: 0s - loss: 0.3433 - accuracy: 0.8619INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.3433 - accuracy: 0.8619 - val_loss: 0.3087 - val_accuracy: 0.8777 - lr: 0.0010
Epoch 3/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3284 - accuracy: 0.8681 - val_loss: 0.3625 - val_accuracy: 0.8441 - lr: 0.0010
Epoch 4/20
568/571 [============================>.] - ETA: 0s - loss: 0.3107 - accuracy: 0.8774INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


...

571/571 [==============================] - 6s 10ms/step - loss: 0.2864 - accuracy: 0.8877 - val_loss: 0.2733 - val_accuracy: 0.8959 - lr: 0.0010
Epoch 8/20
567/571 [============================>.] - ETA: 0s - loss: 0.2776 - accuracy: 0.8907INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.2779 - accuracy: 0.8906 - val_loss: 0.2694 - val_accuracy: 0.8986 - lr: 0.0010
Epoch 9/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2753 - accuracy: 0.8935 - val_loss: 0.2813 - val_accuracy: 0.8873 - lr: 0.0010
Epoch 10/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2682 - accuracy: 0.8974 - val_loss: 0.2686 - val_accuracy: 0.8959 - lr: 0.0010
Epoch 11/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2632 - accuracy: 0.8979 - val_loss: 0.2721 - val_accuracy: 0.8927 - lr: 0.0010
Epoch 12/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2559 - accuracy: 0.9004 - val_loss: 0.2754 - val_accuracy: 0.8937 - lr: 0.0010
Epoch 13/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2547 - accuracy: 0.9021 - val_loss: 0.2751 - val_accuracy: 0.8937 - lr: 0.0010
Epoch 14/20
571/571 [==============================] - 5s 8ms/step - loss: 0

2023-09-19 04:51:22.678738: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 0s 2ms/step
PLeccNET
TP: 2224 FP: 282 TN: 1738 FN: 261
ACC: 0.8794672586015538 F1: 0.8912041675015027 MCC: 0.756134932336595


In [7]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,1])
        elif (j == 'T'):
            dd.append([0,2])
        elif (j == 'G'):
            dd.append([0,3])
        elif (j == 'C'):
            dd.append([0,4])
        else:
            dd.append([0,0])
    while(len(dd) < 500):
        dd.append([0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,2),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(40544, 500, 2)
(4505, 500, 2)
Epoch 1/20
568/571 [============================>.] - ETA: 0s - loss: 0.6175 - accuracy: 0.6253INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 9s 11ms/step - loss: 0.6173 - accuracy: 0.6254 - val_loss: 0.5330 - val_accuracy: 0.7255 - lr: 0.0010
Epoch 2/20
568/571 [============================>.] - ETA: 0s - loss: 0.5210 - accuracy: 0.7320INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.5209 - accuracy: 0.7322 - val_loss: 0.4697 - val_accuracy: 0.7800 - lr: 0.0010
Epoch 3/20
571/571 [==============================] - ETA: 0s - loss: 0.4732 - accuracy: 0.7697INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


...

571/571 [==============================] - 6s 10ms/step - loss: 0.3651 - accuracy: 0.8409 - val_loss: 0.3333 - val_accuracy: 0.8713 - lr: 0.0010
Epoch 8/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3573 - accuracy: 0.8494 - val_loss: 0.3652 - val_accuracy: 0.8496 - lr: 0.0010
Epoch 9/20
569/571 [============================>.] - ETA: 0s - loss: 0.3444 - accuracy: 0.8552INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.3442 - accuracy: 0.8552 - val_loss: 0.3085 - val_accuracy: 0.8826 - lr: 0.0010
Epoch 10/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3406 - accuracy: 0.8578 - val_loss: 0.3062 - val_accuracy: 0.8819 - lr: 0.0010
Epoch 11/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3334 - accuracy: 0.8604 - val_loss: 0.3036 - val_accuracy: 0.8794 - lr: 0.0010
Epoch 12/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3299 - accuracy: 0.8622 - val_loss: 0.3000 - val_accuracy: 0.8826 - lr: 0.0010
Epoch 13/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3224 - accuracy: 0.8673 - val_loss: 0.3115 - val_accuracy: 0.8703 - lr: 0.0010
Epoch 14/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3183 - accuracy: 0.8689 - val_loss: 0.3830 - val_accuracy: 0.8180 - lr: 0.0010
Epoch 15/20
571/571 [==============================] - 5s 8ms/step - loss: 

2023-09-19 04:55:00.433102: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 0s 2ms/step
PLeccNET
TP: 2161 FP: 345 TN: 1745 FN: 254
ACC: 0.8670366259711432 F1: 0.8782767730136151 MCC: 0.7325043711788962


In [8]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([1])
        elif (j == 'T'):
            dd.append([-1])
        elif (j == 'G'):
            dd.append([2])
        elif (j == 'C'):
            dd.append([-2])
        else:
            dd.append([0])
    while(len(dd) < 500):
        dd.append([0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,1),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(40544, 500, 1)
(4505, 500, 1)
Epoch 1/20
570/571 [============================>.] - ETA: 0s - loss: 0.5248 - accuracy: 0.7368INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 9s 11ms/step - loss: 0.5248 - accuracy: 0.7368 - val_loss: 0.4142 - val_accuracy: 0.8047 - lr: 0.0010
Epoch 2/20
571/571 [==============================] - 5s 8ms/step - loss: 0.4327 - accuracy: 0.7984 - val_loss: 0.4593 - val_accuracy: 0.7716 - lr: 0.0010
Epoch 3/20
568/571 [============================>.] - ETA: 0s - loss: 0.3838 - accuracy: 0.8291INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.3838 - accuracy: 0.8291 - val_loss: 0.3281 - val_accuracy: 0.8668 - lr: 0.0010
Epoch 4/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3553 - accuracy: 0.8493 - val_loss: 0.3315 - val_accuracy: 0.8604 - lr: 0.0010
Epoch 5/20
568/571 [============================>.] - ETA: 0s - loss: 0.3408 - accuracy: 0.8568INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


...

571/571 [==============================] - 6s 10ms/step - loss: 0.3089 - accuracy: 0.8729 - val_loss: 0.2855 - val_accuracy: 0.8866 - lr: 0.0010
Epoch 11/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3047 - accuracy: 0.8756 - val_loss: 0.3778 - val_accuracy: 0.8409 - lr: 0.0010
Epoch 12/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3027 - accuracy: 0.8769 - val_loss: 0.3357 - val_accuracy: 0.8644 - lr: 0.0010
Epoch 13/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2977 - accuracy: 0.8791 - val_loss: 0.3060 - val_accuracy: 0.8789 - lr: 0.0010
Epoch 14/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2903 - accuracy: 0.8836 - val_loss: 0.2911 - val_accuracy: 0.8838 - lr: 0.0010
Epoch 15/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2897 - accuracy: 0.8824 - val_loss: 0.3810 - val_accuracy: 0.8451 - lr: 0.0010
Epoch 16/20
566/571 [============================>.] - ETA: 0s - loss: 0.28

INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.2849 - accuracy: 0.8865 - val_loss: 0.2868 - val_accuracy: 0.8880 - lr: 0.0010
Epoch 17/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2764 - accuracy: 0.8895 - val_loss: 0.2930 - val_accuracy: 0.8871 - lr: 0.0010
Epoch 18/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2796 - accuracy: 0.8891 - val_loss: 0.3821 - val_accuracy: 0.8493 - lr: 0.0010
Epoch 19/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2728 - accuracy: 0.8913 - val_loss: 0.3270 - val_accuracy: 0.8705 - lr: 0.0010
Epoch 20/20
571/571 [==============================] - 5s 8ms/step - loss: 0.2690 - accuracy: 0.8930 - val_loss: 0.3725 - val_accuracy: 0.8592 - lr: 0.0010


2023-09-19 04:58:37.250965: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 0s 2ms/step
PLeccNET
TP: 2169 FP: 337 TN: 1764 FN: 235
ACC: 0.8730299667036626 F1: 0.8835030549898167 MCC: 0.7448989262957463


In [9]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([1])
        elif (j == 'T'):
            dd.append([2])
        elif (j == 'G'):
            dd.append([3])
        elif (j == 'C'):
            dd.append([4])
        else:
            dd.append([0])
    while(len(dd) < 500):
        dd.append([0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,1),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(40544, 500, 1)
(4505, 500, 1)
Epoch 1/20
569/571 [============================>.] - ETA: 0s - loss: 0.6136 - accuracy: 0.6279INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 9s 11ms/step - loss: 0.6135 - accuracy: 0.6279 - val_loss: 0.5432 - val_accuracy: 0.6974 - lr: 0.0010
Epoch 2/20
570/571 [============================>.] - ETA: 0s - loss: 0.5008 - accuracy: 0.7481INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.5008 - accuracy: 0.7481 - val_loss: 0.4281 - val_accuracy: 0.8076 - lr: 0.0010
Epoch 3/20
568/571 [============================>.] - ETA: 0s - loss: 0.4465 - accuracy: 0.7882INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


...

571/571 [==============================] - 6s 10ms/step - loss: 0.3462 - accuracy: 0.8533 - val_loss: 0.3100 - val_accuracy: 0.8755 - lr: 0.0010
Epoch 14/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3333 - accuracy: 0.8603 - val_loss: 0.3843 - val_accuracy: 0.8180 - lr: 0.0010
Epoch 15/20
567/571 [============================>.] - ETA: 0s - loss: 0.3337 - accuracy: 0.8608INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.3337 - accuracy: 0.8608 - val_loss: 0.3033 - val_accuracy: 0.8811 - lr: 0.0010
Epoch 16/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3330 - accuracy: 0.8587 - val_loss: 0.3102 - val_accuracy: 0.8735 - lr: 0.0010
Epoch 17/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3325 - accuracy: 0.8599 - val_loss: 0.3512 - val_accuracy: 0.8397 - lr: 0.0010
Epoch 18/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3269 - accuracy: 0.8643 - val_loss: 0.3087 - val_accuracy: 0.8718 - lr: 0.0010
Epoch 19/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3215 - accuracy: 0.8672 - val_loss: 0.3163 - val_accuracy: 0.8666 - lr: 0.0010
Epoch 20/20
571/571 [==============================] - 5s 8ms/step - loss: 0.3230 - accuracy: 0.8636 - val_loss: 0.3946 - val_accuracy: 0.8160 - lr: 0.0010


2023-09-19 05:11:08.340047: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 0s 2ms/step
PLeccNET
TP: 2141 FP: 365 TN: 1777 FN: 222
ACC: 0.869700332963374 F1: 0.8794413637297186 MCC: 0.7394622074164671


In [10]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D,Bidirectional, GRU,Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge

lu_model = 'bianma'
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/'+lu_model+'/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)
f = open('./ecc/arabidopsis/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0])
        elif (j == 'T'):
            dd.append([1])
        elif (j == 'G'):
            dd.append([1])
        elif (j == 'C'):
            dd.append([0])
        else:
            dd.append([0])
    while(len(dd) < 500):
        dd.append([0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)


inputs1 = Input(shape=(500,1),name='1st_input')

x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)

x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

CNN_model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    # hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = cross_entropy_loss
    
    return total_loss




CNN_model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



CNN_history = CNN_model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


CNN_model.load_weights('./model/'+lu_model+'/bestmodel')
result = open('./arabidopsis_roc_data','a')
pl = CNN_model.predict(test_data)
re = []
for i in range(len(pl)):
    re.append(round(pl[i][0],4))

re = list(map(lambda x: str(x), re))
re = 'p_CNN_model'+','+','.join(re)
# print(re)
result.write(re+'\n')
test_re = list(map(lambda x: str(x), test_label))
test_re = 'test_CNN_model'+','+','.join(test_re)
# print(test_re)

result.write(test_re+'\n')

result.close()
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(40544, 500, 1)
(4505, 500, 1)
Epoch 1/20
567/571 [============================>.] - ETA: 0s - loss: 0.6298 - accuracy: 0.5826INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 8s 11ms/step - loss: 0.6298 - accuracy: 0.5825 - val_loss: 0.6100 - val_accuracy: 0.5891 - lr: 0.0010
Epoch 2/20
568/571 [============================>.] - ETA: 0s - loss: 0.6149 - accuracy: 0.5902INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.6148 - accuracy: 0.5904 - val_loss: 0.6092 - val_accuracy: 0.5965 - lr: 0.0010
Epoch 3/20
566/571 [============================>.] - ETA: 0s - loss: 0.6130 - accuracy: 0.5933INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


...

571/571 [==============================] - 6s 10ms/step - loss: 0.5994 - accuracy: 0.6236 - val_loss: 0.5991 - val_accuracy: 0.6249 - lr: 0.0010
Epoch 16/20
571/571 [==============================] - 5s 8ms/step - loss: 0.5990 - accuracy: 0.6247 - val_loss: 0.5998 - val_accuracy: 0.6187 - lr: 0.0010
Epoch 17/20
568/571 [============================>.] - ETA: 0s - loss: 0.5982 - accuracy: 0.6272INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bianma/bestmodel/assets


571/571 [==============================] - 6s 10ms/step - loss: 0.5981 - accuracy: 0.6274 - val_loss: 0.5993 - val_accuracy: 0.6303 - lr: 0.0010
Epoch 18/20
571/571 [==============================] - 5s 8ms/step - loss: 0.5985 - accuracy: 0.6251 - val_loss: 0.6019 - val_accuracy: 0.6175 - lr: 0.0010
Epoch 19/20
571/571 [==============================] - 5s 8ms/step - loss: 0.5975 - accuracy: 0.6273 - val_loss: 0.5992 - val_accuracy: 0.6195 - lr: 0.0010
Epoch 20/20
571/571 [==============================] - 5s 8ms/step - loss: 0.5969 - accuracy: 0.6290 - val_loss: 0.6017 - val_accuracy: 0.6224 - lr: 0.0010


2023-09-19 05:14:49.090877: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bianma/bestmodel: FAILED_PRECONDITION: model/bianma/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


141/141 [==============================] - 0s 2ms/step
PLeccNET
TP: 1474 FP: 1032 TN: 1255 FN: 744
ACC: 0.6057713651498335 F1: 0.6240474174428451 MCC: 0.21465517450276503
